# Reproducing Figures
This notebook demonstrates the steps required to reproduce the figures presented in the paper. Follow the instructions below to ensure the environment and data are properly set up.

# Notes for Reproducibility
+ Ensure that the same data and parameter values are used as described in the model

# 1. Prerequisites
**This code block will use the required packages needed to make the figures, and the julia file for processing data and visualization**

In [ ]:
# Import necessary packages
using HDF5
using PyPlot
using Statistics
# Import necessary julia file
include("PostProcess.jl");

# 2. Data Preparation
**The following code will require the parameters used in the model to be defined, and the path to the data**
- range of ice shell thickenss
- range of wavelength
- amplitude
- gravity of icy moon
- data path

Note: the units must match the parameter defined in model

In [ ]:
# Define your ice shell thickness range
ice_shell_thickness_range = range(10.0,60.0,15)
# Define your wavelength range
wavelength_range = range(5.0,300.0,14)  
# Define the amplitude value
amplitude = 20.0  
# Define gravity
gravity = 0.113 
# Define the data file path
folder_name = "Enceladus"
# Define the output file path
output_path = folder_name*"_combined_output.hdf5";

# 3. Post-process Data 
**The following code will process the data for visualization**

In [ ]:
combine_hdf5_files(ice_shell_thickness_range,wavelength_range,amplitude,gravity,output_path,folder_name)
fname = output_path
fid = h5open(fname,"r")
# Reading Data from file
modeldata = fid["Combined Model Run"];
# Uncomment the following code if you wish to display the contents 
# for obj in modeldata
#     data = read(obj)
#     println(obj)
#     display(data)
# end
# Accessing contents and defining contents needed
Wavelength = read(modeldata,"Wavelength");
Ice_shell = read(modeldata, "Ice Shell Thickness");
Time_halfspace = read(modeldata,"Viscous Relaxation Time(Half-Space)");
Time_rel = read(modeldata,"Viscous Relaxation Time(Model)");
Time_thickening = read(modeldata, "Thickening Time");
Time_thickening_fit = read(modeldata, "Fitted Thickening Time");
Time_rel_fit = read(modeldata, "Fitted Viscous Relaxation Time");
# Close file
close(fid)
# Defining t'
t_prime = Time_rel_fit./Time_thickening_fit;

# 4. Generating Figures
**The following code will make the figures for $t'$, $t_{relaxation}$, and $t_{thickening}$**

In [ ]:
# Importing matplotlib variables
cm = PyPlot.matplotlib[:cm]
LogNorm = PyPlot.matplotlib[:colors][:LogNorm]
ticker = PyPlot.matplotlib[:ticker]

### Plot for t' ###
## starts here ##
figure()
# Smooth-looking contour plot in the background
levs = exp10.(range(-5,1,length=128))
cs1=contourf(Ice_shell/1e3,(2*Wavelength)/1e3,t_prime,levs,norm=LogNorm(),vmin=1e-5,vmax=1e1,cmap=cm.RdYlBu_r)
levs1 = exp10.([-5,-4,-3,-2,-1,0,1])
# Adding contour lines in the foreground
cs = contour(Ice_shell/1e3,(2*Wavelength)/1e3,t_prime,levs1,norm=LogNorm(),colors="k")
clabel(cs,inline=true,fmt="%2.e",colors="k",fontsize=12)
title("Enceladus",fontsize=14)
colorbar(cs1,ticks=levs1,label=L"\log_{10}(t')")
gca().yaxis.set_label_position("left")
gca().set_ylabel(L"\lambda"*" (km)",fontsize=14)
gca().set_xlabel("H (km)",fontsize=14,rotation=0)
# Highlight region with a square
x0,y0 = 10,600  # replace with the coordinates of the bottom-left corner
width,height = 8,-100  # replace with the dimensions of the square
# Making square
gca().add_patch(PyPlot.matplotlib[:patches][:Rectangle](
        (x0, y0),
        width,
        height,
        fill=true,
        facecolor="gray",
        alpha=0.7,
        linewidth=5))
# Making texts
annotate("SPT-like", 
        xy=(x0,y0), 
        xytext=(x0+8,y0-50), 
        fontsize=16, 
        color="gray")
annotate("(a)", 
        xy=(x0,y0), 
        xytext=(x0-10,y0), 
        fontsize=14, 
        color="black")
show()
## ends here ##

### Plot for viscous relaxation ###
## starts here ##
figure()
# smooth background contour plot:
levs = exp10.(range(2, stop=7, length=100))
cs = contourf(Ice_shell/1e3,(2*Wavelength)/1e3,Time_rel_fit,levs,norm=LogNorm(),vmin=1e2,vmax=1e7)
# contour levels to highlight
levs1 = exp10.([2,3,4,5,6,7])
cs1 = contour(Ice_shell/1e3,(2*Wavelength)/1e3,Time_rel_fit,levs1,norm=LogNorm(),colors="k")
# scientific notation formatter
fmt = PyPlot.matplotlib[:ticker][:LogFormatterSciNotation]()
labels = Dict()
for level in cs1.levels
    labels[level] = fmt(level)
end
clabel(cs1,inline=true,fmt=labels,colors="k",fontsize=12)
colorbar(cs,ticks=levs1,label="Relaxation Timescale (yr)")
title(L"t_{relaxation}",fontsize=14)
gca().yaxis.set_label_position("left")
gca().set_ylabel(L"\lambda"*" (km)",fontsize=14)
gca().set_xlabel("H (km)",fontsize=14,rotation=0)
# Making text
annotate("(a)", 
        xy=(x0,y0), 
        xytext=(x0-10,y0), 
        fontsize=14, 
        color="black")
show()
## ends here ##

### Plot for thickening ###
## starts here ##
figure()
# Define the 3 levels to highlight
highlight_levels = [0.5e7,1.0e7,1.5e7,2.0e7]
# Plot all 32 levels with muted lines
cs = contour(Ice_shell/1e3, (2*Wavelength)/1e3, Time_thickening_fit,levels=100)
# Plot only the 3 highlight levels with a thicker line or different color
highlight_cs = contour(Ice_shell/1e3,(2*Wavelength)/1e3,Time_thickening_fit, colors="k",levels=highlight_levels)
# Fill the contours with the colormap
contourf(cs,cmap=get_cmap("viridis"))
# Add title and contour labels
title(L"t_{thickening}",fontsize=14)
clabel(highlight_cs,inline=true,fmt="%1.0f",colors="k",fontsize=12)  # Labels for highlighted levels
# Add the colorbar
cbar = colorbar(label="Thickening Timescale (yr)", location="right")
# Set the ScalarFormatter for the colorbar to use scientific notation
formatter = ticker.ScalarFormatter(useMathText=true)
formatter.set_scientific(true)
formatter.set_powerlimits((-2, 2))
formatter.set_useOffset(false)
cbar.ax.yaxis.set_major_formatter(formatter)
# Set labels and formatting
gca().yaxis.set_label_position("left")
gca().set_ylabel(L"\lambda"*" (km)", fontsize=14)
gca().set_xlabel("H (km)", fontsize=14, rotation=0)

# Making text
annotate("(b)", 
        xy=(x0,y0), 
        xytext=(x0-10,y0), 
        fontsize=14, 
        color="black")
show()
## ends here ##